In [20]:
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import glob
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


In [21]:
## for Sklearn LDA
IdtoWord = {}
WordtoId = {}
vocab = set()
with open("DataPreprocess/wordToIDtoy20.txt") as f:
    for line in f:
        line = line.split()
        IdtoWord[int(line[1])] = line[0]
        WordtoId[line[0]] = int(line[1])
        vocab.add(line[0])
    
D = 20
V = len(vocab)
doc_word_matrix = np.zeros(shape = (D, V), dtype = np.int64)

In [22]:
txt_files = glob.glob("DataPreprocess/nipstxt/nipstoy20/short*.txt")

gensim_texts = [] ## for gensim
count = 0 #count number of documents
for file in (txt_files):
    with open(file, 'rt', encoding="ISO-8859-1") as f:
        words_j = f.readline().split()
        gensim_texts.append(words_j)
        for words_ij in words_j:
            col_index = WordtoId[words_ij]
            doc_word_matrix[count][col_index]+=1
        count += 1

In [23]:
### Gensim LDA
id2word = corpora.Dictionary(gensim_texts)
texts = gensim_texts
corpus = [id2word.doc2bow(text) for text in texts]
lda_gensim = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

In [24]:
pprint(lda_gensim.print_topics())
doc_lda = lda_gensim[corpus]

[(0,
  '0.014*"language" + 0.014*"recurrent" + 0.014*"stimulus" + '
  '0.014*"processing" + 0.009*"method" + 0.009*"pfms" + 0.009*"orientation" + '
  '0.009*"abstract" + 0.009*"onset" + 0.009*"neural"'),
 (1,
  '0.019*"similarity" + 0.015*"rules" + 0.015*"law" + 0.015*"channels" + '
  '0.010*"models" + 0.010*"connections" + 0.010*"perception" + '
  '0.010*"stimulus" + 0.010*"feedback" + 0.010*"paper"'),
 (2,
  '0.002*"neurons" + 0.001*"results" + 0.001*"lights" + 0.001*"abstract" + '
  '0.001*"similarity" + 0.001*"ep" + 0.001*"two" + 0.001*"rules" + '
  '0.001*"rule" + 0.001*"layer"'),
 (3,
  '0.017*"show" + 0.017*"model" + 0.017*"coefficients" + 0.017*"real" + '
  '0.017*"functions" + 0.017*"basis" + 0.017*"valued" + 0.009*"image" + '
  '0.009*"abstract" + 0.009*"spiking"'),
 (4,
  '0.033*"lights" + 0.028*"vr" + 0.028*"ep" + 0.022*"stop" + 0.012*"results" + '
  '0.011*"environment" + 0.011*"recognition" + 0.011*"virtual" + 0.011*"show" '
  '+ 0.011*"slow"'),
 (5,
  '0.018*"spatial" + 

In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_gensim.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=gensim_texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.831340711469685


/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 


Coherence Score:  0.4585185063674423


In [26]:
## Sklearn LDA
lda_model = LatentDirichletAllocation(n_components=10, random_state=0)
lda_model.fit(doc_word_matrix)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [27]:
## get top 15 words per topic
id_topic = lda_model.fit_transform(doc_word_matrix)
topic_words = {}
n_top_words = 15

for topic, beta in enumerate(lda_model.components_):
    word_idx = np.argsort(beta)[::-1][:n_top_words]
    topic_words['topic: '+str(topic)] = [IdtoWord[i] for i in word_idx]

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [28]:
for key,value in topic_words.items():
    print (key)
    for word in value:
        print (word)

topic: 0
lights
attention
visual
ep
vr
language
languages
system
stop
environment
neural
different
search
show
dynamic
topic: 1
studied
immersive
representations
domain
pattern
search
argued
computing
arbors
provides
simple
less
explicit
magnitude
higher
topic: 2
spiking
synaptic
neurons
abstract
distributions
learning
grouping
cell
synchrony
assembly
node
valued
functions
coefficients
widths
topic: 3
tasks
class
classification
recognition
attentional
model
attention
solution
response
guided
results
levels
parity
expert
abstract
topic: 4
spatial
information
neurons
networks
attractor
layout
direct
temporal
input
output
attractors
given
abstract
encoding
formulation
topic: 5
learning
hebbian
effective
synaptic
efcacies
network
synapses
associative
neuronal
normalization
capacity
requirement
memory
incoming
also
topic: 6
precise
first
time
present
recurrent
related
combinatorial
objective
essentially
perception
expert
assisted
term
lateral
incoming
topic: 7
stimulus
context
channels
law
